In [1]:
!pip install nltk
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

### Get some sample product_ids

In [2]:
import pandas as pd
import numpy as np
import pdb
from tqdm import tqdm
import nltk
import string
import pickle

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [3]:
# Only for the first time run below code
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
metadata_filename = 'Movies_dataset.csv'
metadata = pd.read_csv(metadata_filename)
metadata.sample(5)

,Unnamed: 0,movieId,imdbId,tmdbId,title,overview,budget,genres,production_companies,production_countries,release_date,runtime,spoken_languages,keywords,cast,crew,final_poster_url
29353,30182,133171,3321254,227200.0,Barbie & Her Sisters in A Pony Tale,Barbie and her sisters set off on a Swiss adve...,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2013-10-07,72.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 2284, 'name': 'horseback riding'}, {'i...","[{'cast_id': 1, 'character': 'Barbie', 'credit...","[{'credit_id': '52fe4e989251416c7515bd35', 'de...",NF
37797,39992,158318,1869360,36607.0,Dave Attell's Insomniac Tour: Uncensored!,Inspired by Dave Attell's popular Comedy Centr...,0.0,"[{'id': 35, 'name': 'Comedy'}]",[],[],2005-02-04,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 3485, 'name': 'comedian'}, {'id': 9716...","[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5304cf58c3a3683a991cf6bd', 'de...",NF
34569,35635,148687,2414166,137366.0,Led Zeppelin - Celebration Day,"On December 10, 2007, Led Zeppelin took the st...",0.0,"[{'id': 10402, 'name': 'Music'}]",[],"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",2012-10-17,124.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 5288, 'name': 'rock'}, {'id': 18001, '...","[{'cast_id': 1, 'character': 'Himself (Vocals ...","[{'credit_id': '52fe4c2fc3a368484e1ab98d', 'de...",NF
41335,43655,168330,5710514,425591.0,I Don't Feel at Home in This World Anymore,"When a depressed woman is burglarized, she fin...",0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...","[{'name': 'Film Science', 'id': 1976}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-01-19,96.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1585, 'name': 'snake'}, {'id': 1930, '...","[{'cast_id': 0, 'character': 'Ruth Kimke', 'cr...","[{'credit_id': '5862d30ac3a36852ba002167', 'de...",https://image.tmdb.org/t/p/w500/1stdUlXBc3nxqh...
29707,30542,134164,75656,86633.0,The Alien Factor,A spaceship containing specimens for an interg...,0.0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...","[{'name': 'Cinemagic Inc.', 'id': 13757}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1978-05-12,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 9951, 'name': 'alien'}]","[{'cast_id': 2, 'character': 'Ben Zachary', 'c...","[{'credit_id': '52fe49939251416c910b1ba5', 'de...",NF


In [5]:
mIDs = metadata.loc[:,'movieId'].values
print(mIDs.shape)

(44434,)


In [6]:
# Check whether movieIds are repeated
u, c = np.unique(mIDs, return_counts=True)
dup = u[c > 1]
dup

array([], dtype=int64)

In [7]:
metadata.columns

Index(['Unnamed: 0', 'movieId', 'imdbId', 'tmdbId', 'title', 'overview',
       'budget', 'genres', 'production_companies', 'production_countries',
       'release_date', 'runtime', 'spoken_languages', 'keywords', 'cast',
       'crew', 'final_poster_url'],
      dtype='object')

In [8]:
def create_description(row):
    genres_list = [genre['name'] for genre in eval(row['genres'])]
    comp_list = [comp['name'] for comp in eval(row['production_companies'])]
    country_list = [country['name'] for country in eval(row['production_countries'])]
    lang_list = [lang['name'] for lang in eval(row['spoken_languages'])]
    kw_list = [kw['name'] for kw in eval(row['keywords'])]
    cast_list = [actor['name'] for actor in eval(row['cast'])]
    crew_list = [crew['name'] for crew in eval(row['crew'])]

    description = f"Title: {row['title']}, Overview: {row['overview']}, Genres: {genres_list}, Production companies: {comp_list}, "
    description += f"Production countries: {country_list}, Spoken languages: {lang_list}, Release date: {row['release_date']}, "
    description += f"Keywords: {kw_list}, Cast: {cast_list}, Crew: {crew_list}"
    return description

create_description(metadata.iloc[3025,:])

"Title: Fantasia 2000, Overview: Blending lively music and brilliant animation, this sequel to the original 'Fantasia' restores 'The Sorcerer's Apprentice' and adds seven new shorts., Genres: ['Animation', 'Family', 'Music'], Production companies: ['Walt Disney Pictures'], Production countries: ['United States of America'], Spoken languages: ['English'], Release date: 1999-12-17, Keywords: ['orchestra', 'musical segments'], Cast: ['Steve Martin', 'Itzhak Perlman', 'Quincy Jones', 'Bette Midler', 'James Earl Jones', 'Penn Jillette', 'Teller', 'James Levine', 'Angela Lansbury', 'Wayne Allwine', 'Tony Anselmo', 'Russi Taylor'], Crew: ['Donald W. Ernst', 'Ludwig van Beethoven', 'James Algar', 'Ruth Lambert', 'Mary Hidalgo', 'Don Hahn', 'George Gershwin', 'Lois Freeman-Fox', 'Gaëtan Brizzi', 'Paul Brizzi', 'Eric Goldberg', 'Andreas Deja', 'Roy Edward Disney', 'Hendel Butoy', 'Francis Glebas', 'Pixote Hunt', 'Mel Metcalfe', 'Terry Porter', 'Shawn Murphy', 'John Brian King', 'Peter Gelb', 'Le

In [9]:
metadata['description'] = metadata.apply(create_description, axis=1)

In [10]:
sample_desc = metadata.description.tolist()[0]
sample_desc

'Title: Toy Story, Overview: Led by Woody, Andy\'s toys live happily in his room until Andy\'s birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy\'s heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences., Genres: [\'Animation\', \'Comedy\', \'Family\'], Production companies: [\'Pixar Animation Studios\'], Production countries: [\'United States of America\'], Spoken languages: [\'English\'], Release date: 1995-10-30, Keywords: [\'jealousy\', \'toy\', \'boy\', \'friendship\', \'friends\', \'rivalry\', \'boy next door\', \'new toy\', \'toy comes to life\'], Cast: [\'Tom Hanks\', \'Tim Allen\', \'Don Rickles\', \'Jim Varney\', \'Wallace Shawn\', \'John Ratzenberger\', \'Annie Potts\', \'John Morris\', \'Erik von Detten\', \'Laurie Metcalf\', \'R. Lee Ermey\', \'Sarah Freeman\', \'Penn Jillette\'], Crew: [\'John Lasseter\', \'Joss Whedon\', \'Andrew Stanton\', 

In [11]:
def text_preprocessing(text: str):

    # Lowercase
    text = text.lower()
    # Remove Punctuation
    # text = "".join([char for char in text if char not in string.punctuation])
    words = word_tokenize(text)
    # Remove Stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    ## Stemming
    # porter = PorterStemmer()
    # stemmed = [porter.stem(word) for word in filtered_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized)

In [12]:
text_preprocessing(sample_desc)

"title : toy story , overview : led woody , andy 's toy live happily room andy 's birthday brings buzz lightyear onto scene . afraid losing place andy 's heart , woody plot buzz . circumstance separate buzz woody owner , duo eventually learns put aside differences. , genre : [ 'animation ' , 'comedy ' , 'family ' ] , production company : [ 'pixar animation studio ' ] , production country : [ 'united state america ' ] , spoken language : [ 'english ' ] , release date : 1995-10-30 , keywords : [ 'jealousy ' , 'toy ' , 'boy ' , 'friendship ' , 'friends ' , 'rivalry ' , 'boy next door ' , 'new toy ' , 'toy come life ' ] , cast : [ 'tom hank ' , 'tim allen ' , 'don rickles ' , 'jim varney ' , 'wallace shawn ' , 'john ratzenberger ' , 'annie potts ' , 'john morris ' , 'erik von detten ' , 'laurie metcalf ' , ' r . lee ermey ' , 'sarah freeman ' , 'penn jillette ' ] , crew : [ 'john lasseter ' , 'joss whedon ' , 'andrew stanton ' , 'joel cohen ' , 'alec sokolow ' , 'bonnie arnold ' , 'ed catm

In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [14]:
list(metadata.description)

['Title: Toy Story, Overview: Led by Woody, Andy\'s toys live happily in his room until Andy\'s birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy\'s heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences., Genres: [\'Animation\', \'Comedy\', \'Family\'], Production companies: [\'Pixar Animation Studios\'], Production countries: [\'United States of America\'], Spoken languages: [\'English\'], Release date: 1995-10-30, Keywords: [\'jealousy\', \'toy\', \'boy\', \'friendship\', \'friends\', \'rivalry\', \'boy next door\', \'new toy\', \'toy comes to life\'], Cast: [\'Tom Hanks\', \'Tim Allen\', \'Don Rickles\', \'Jim Varney\', \'Wallace Shawn\', \'John Ratzenberger\', \'Annie Potts\', \'John Morris\', \'Erik von Detten\', \'Laurie Metcalf\', \'R. Lee Ermey\', \'Sarah Freeman\', \'Penn Jillette\'], Crew: [\'John Lasseter\', \'Joss Whedon\', \'Andrew Stanton\',

In [15]:
encoded_data = model.encode(metadata.description.tolist(), device='cuda')
encoded_data = np.asarray(encoded_data.astype('float32'))


In [16]:
encoded_title_data = model.encode(metadata.title.tolist(), device='cuda')
encoded_title_data = np.asarray(encoded_title_data.astype('float32'))


In [17]:
with open('encodings_description.pickle','wb') as file:
    pickle.dump(encoded_data, file, protocol=pickle.HIGHEST_PROTOCOL)

with open('encodings_title.pickle','wb') as file:
    pickle.dump(encoded_title_data, file, protocol=pickle.HIGHEST_PROTOCOL)

with open('model.pickle','wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
encoded_data.shape

(44434, 768)

In [ ]:
with open('encodings_descriiption.pickle','rb') as file:
    encoding_loaded = pickle.load(file)

with open('model.pickle','rb') as file:
    model_loaded = pickle.load(file)

In [19]:
def find_similar_items(query: str, model: SentenceTransformer, encodings: np.array, top_n: int = 5):
    query_vector = model.encode([query])
    # print(query_vector)
    similarity = np.dot(encodings,query_vector.T)
    # print(similarity)
    top_items = similarity.flatten().argsort()[-top_n:][::-1]
    # print(top_items)
    # return metadata['asin'].iloc[top_items]
    return top_items

In [20]:
top_n = 10
query = "A love story between a prince and a normal girl"

top_items = find_similar_items(query=query, model=model, encodings=encoded_data, top_n=top_n)
metadata['movieId'].iloc[top_items]

40253    165401
36704    154822
36536    154400
44132    175507
36619    154594
27913    128594
21282    104335
19363     96911
31124    138414
7202       7375
Name: movieId, dtype: int64

In [21]:
top_n = 10
query = "adventure genre with title jumanji"

top_items = find_similar_items(query=query, model=model, encodings=encoded_data, top_n=top_n)
metadata['movieId'].iloc[top_items]

25422    119830
22551    109006
36654    154664
25456    120110
15429     79166
18433     92633
1             2
40563    166297
8829      26510
33742    146062
Name: movieId, dtype: int64

In [22]:
top_n = 10
query = "adventure genre with title jumanji"

top_items = find_similar_items(query=query, model=model, encodings=encoded_title_data, top_n=top_n)
metadata['movieId'].iloc[top_items]

1             2
16755     85387
35573    151959
41785    169450
36361    153887
22732    109677
33686    145931
9416      27787
19936     99180
13024     62974
Name: movieId, dtype: int64

In [23]:
num_duplicates = metadata.duplicated(subset=metadata.columns.difference(['crew'])).sum()

print("Number of duplicated rows:", num_duplicates)

Number of duplicated rows: 0
